In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

import os
env = os.environ.copy()
env["OLLAMA_HOST"] = "0.0.0.0"
env["OLLAMA_ORIGINS"] = "*"

import subprocess
def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"], env=env)

import threading
thread = threading.Thread(target=run_ollama_serve)
thread.start()

import time
time.sleep(5)

In [ ]:
!ollama pull gpt-oss:20b
!curl http://localhost:11434/api/generate -d '{ "model": "gpt-oss:20b", "prompt": "Who are you?", "stream": false}'

#curl http://n2.ckey.vn:2421/api/generate -d '{ "model": "gpt-oss:20b", "prompt": "Who are you?", "stream": false}'


{"model":"gpt-oss:20b","created_at":"2025-09-18T06:42:32.052063248Z","response":"I’m ChatGPT, a large language‑model created by OpenAI.  \n- **What I am**: a machine‑learning system that processes text, understands context, and generates responses in natural language.  \n- **What I can do**: answer questions, help brainstorm, explain concepts, draft documents, offer creative ideas, simulate conversations, and more.  \n- **How I work**: I’ve been trained on a diverse mix of text (books, articles, websites, etc.) up to my last update in 2024. I don’t have opinions, beliefs, or personal experiences—everything I say comes from patterns in that training data.  \n- **What I’m not**: a human, a real person, or a conscious being. I don’t have feelings, memories, or access to personal data unless it’s shared with me during our chat.  \n\nFeel free to ask me anything you’d like help with!","thinking":"User: \"Who are you?\" They want explanation. Should provide identity, maybe mention I'm ChatG

In [ ]:
!pip install colab-xterm
%load_ext colabxterm
%xterm

# TODO: Chạy lệnh dưới để tunnel ra bên ngoài CHÚ Ý: NHẤN PHẢI RỒI CHỌN COPY TRONG MENU, NẾU BẤM CTRL + C SẼ TẮT KẾT NỐI
# ssh -p 443 -R0:localhost:11434 qr@a.pinggy.io

In [ ]:
# Thay "http://localhost:11434" bằng link sinh ra từ pingy.io, ví dụ nếu thấy "http://bsqlv-34-125-123-92.a.free.pinggy.link"
!curl http://tytji-34-124-205-38.a.free.pinggy.link/api/generate -d '{ "model": "gpt-oss:20b", "prompt": "Tell me a joke", "stream": false}'

{"model":"gpt-oss:20b","created_at":"2025-09-18T06:44:03.273097395Z","response":"I’m ChatGPT, a large language model created by OpenAI. I’ve been trained on a wide range of text—from books and websites to articles and code—to help answer questions, generate ideas, write or edit text, and even chat about almost any topic you can think of. I don’t have personal experiences or consciousness; I generate responses by predicting what comes next in a conversation based on patterns I learned during training. My goal is to assist you, so feel free to ask me anything you need help with!","thinking":"The user asks: \"Who are you?\" They are presumably addressing ChatGPT. So answer: I am ChatGPT, a large language model trained by OpenAI. I can provide context. Maybe elaborate. Should mention I am not human, no consciousness. The response should be friendly. They might want to know what capabilities I have. So I should explain I am an AI, large language model, etc.\n\nAlso respond in natural tone. 

In [ ]:
# Test gọi Ollama cục bộ
%pip install ollama
import ollama
response = ollama.chat(model='gpt-oss:20b', messages=[
  {
    'role': 'user',
    'content': 'Tell me a joke',
  },
])
count = response["eval_count"]
time = response["eval_duration"] / 1000000000
avg_tokens = round(count / time, 1)
print(f'{avg_tokens} tokens/s')
print(response['message']['content'])

In [ ]:
# NẾU KHÔNG MUỐN DÙNG PINGGY MÀ DÙNG NGROK THÌ SAO?
#TODO: Vào ngrok.com đăng kí một account để lấy API-KEY
%pip install pyngrok
from pyngrok import ngrok

from google.colab import userdata
ngrok_api_key = userdata.get('ngrok-api-key')
ngrok.set_auth_token(ngrok_api_key)

# ngrok có giới hạn 1 GB/tháng
# zrok giới hạn là 5 GB/ngày
# https://github.com/int11/Kaggle_remote_zrok
# https://www.kaggle.com/code/kayak0/kaggle-zrok


port = 11434 # Mặc định Ollama chạy ở port này
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel {public_url} -> http://127.0.0.1:{port}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!curl https://a7f3566c1126.ngrok-free.app/api/generate -d '{ "model": "gpt-oss:20b", "prompt": "Who are you?", "stream": false}'

{"model":"gpt-oss:20b","created_at":"2025-09-18T07:06:20.684011199Z","response":"I’m ChatGPT, a large‑language model created by OpenAI. I’m trained on a diverse mix of text—from books and articles to websites—so I can help answer questions, brainstorm ideas, explain concepts, and more. I don’t have a personal identity or consciousness; I generate responses based on patterns in the data I was trained on. My goal is to provide useful, accurate, and engaging information, while staying safe and respectful. If you’ve got a specific question or need assistance, just let me know!","thinking":"We need to respond as ChatGPT. The user says \"Who are you?\" They probably want a brief self-introduction: I'm ChatGPT, large language model. Should explain context, capabilities, limitations. Use friendly tone.","done":true,"done_reason":"stop","context":[200006,17360,200008,3575,553,17554,162016,11,261,4410,6439,2359,22203,656,7788,17527,558,87447,100594,25,220,1323,19,12,3218,198,6576,3521,25,220,132

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
!pip install fastapi nest-asyncio pyngrok uvicorn

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root():
    return {'hello': 'world'}

@app.get('/ask')
async def ask(question: str):
    return {
        'question': question,
        'answer': "I don't know"
    }

import threading
import uvicorn

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

print("Server started on http://0.0.0.0:8000")


Server started on http://0.0.0.0:8000


In [ ]:
!curl localhost:8000

INFO:     127.0.0.1:40576 - "GET / HTTP/1.1" 200 OK
{"hello":"world"}

In [ ]:
!curl localhost:8000/ask?question=Tell%20me%20a%20joke

INFO:     127.0.0.1:36330 - "GET /ask?question=Tell%20me%20a%20joke HTTP/1.1" 200 OK
{"question":"Tell me a joke","answer":"I don't know"}

In [ ]:
# Kill the ngrok process, closing all active tunnels
ngrok.kill()